In [1]:
import geopandas as gpd
import pandas as pd
from datetime import datetime, timedelta
from shapely.geometry import Polygon, LineString, MultiPolygon
from shapely.ops import split
import copy
import time
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import copy
import pandas as pd
import sklearn
import pickle
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier
from numpy import loadtxt
from xgboost import XGBClassifier

In [30]:
# burned = gpd.read_file("../data/burned/sampleData.csv")
# unburned = gpd.read_file("../data/unburned/sampleData.csv")

In [ ]:
# neighbors = pd.concat([burned,unburned])
# neighbors

In [42]:
clean_df = pd.DataFrame()
for i in ['CASCADE.csv', 'BALD.csv', 'REDWOOD VALLEY.csv', 'COVE.csv', 'CALDWELL.csv', 'BUTTE.csv', 'BUCK.csv', 'WALKER.csv', 'HAPPY.csv', 'CARR.csv', 'ROCKY.csv', 'OAK.csv', 'KING.csv', 'KINCADE.csv', 'ATLAS.csv', 'CAMP.csv','FRYING PAN.csv']:
    temp = pd.read_csv('../data/rasterCSV/'+str(i))
    names=[i[:len(i)-4]]*temp.shape[0]
    temp['name'] = names
    clean_df = pd.concat([temp,clean_df])

In [7]:
# test = pd.read_csv('../data/rasterCSV/'+'FRYING PAN.csv')

In [33]:
# temp1 = gpd.GeoDataFrame()
# temp2 = gpd.GeoDataFrame()
# for i in np.unique(burned['FIRE_NAME']):
#     temp1 = pd.concat([temp1,burned[burned['FIRE_NAME']==i]])
#     temp2 = pd.concat([temp2,unburned[unburned['FIRE_NAME']==i]])

In [43]:
lands = OneHotEncoder().fit_transform(np.array(clean_df['landCover']).reshape(-1,1))
lands = pd.DataFrame.sparse.from_spmatrix(lands, columns= range(6))
clean_df = clean_df.reset_index().drop(columns='index')
X_train_t = pd.concat([lands, clean_df], axis=1)
clean_df = X_train_t
clean_df = clean_df[[0,
 1,
 2,
 3,
 4,
 5,
 'SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'NDVI',
 'elevation',
 'percent_tree_cover',
'burnSeverity']]

In [44]:
for i in clean_df.columns:
    clean_df[i]=pd.to_numeric(clean_df[i], errors='coerce')
clean_df

<ipython-input-44-62340587ab3f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df[i]=pd.to_numeric(clean_df[i], errors='coerce')


,0,1,2,3,4,5,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,NDVI,elevation,percent_tree_cover,burnSeverity
0,0.0,0.0,1.0,0.0,0.0,0.0,7835,7945,8753,8107,21454,11501,8826,451,558,74,2
1,0.0,0.0,1.0,0.0,0.0,0.0,7794,7904,8654,8034,21443,11210,8669,454,544,76,2
2,0.0,0.0,1.0,0.0,0.0,0.0,7792,7878,8596,8002,20576,11125,8683,439,521,79,2
3,0.0,0.0,1.0,0.0,0.0,0.0,7637,7709,8155,7736,16503,9872,8196,361,513,81,2
4,0.0,0.0,1.0,0.0,0.0,0.0,7608,7659,8065,7658,15235,9619,8132,330,506,82,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4380661,0.0,0.0,0.0,1.0,0.0,0.0,8156,8341,8965,9020,15535,12553,10065,265,338,0,2
4380662,0.0,0.0,0.0,1.0,0.0,0.0,8229,8393,9007,9188,15785,13556,10771,264,332,0,2
4380663,0.0,0.0,0.0,1.0,0.0,0.0,8602,8890,9542,10364,15031,15533,12149,183,327,0,2
4380664,0.0,0.0,0.0,1.0,0.0,0.0,8602,8890,9542,10364,15031,15533,12149,183,318,0,2


In [ ]:
# test = test[['SR_B1',
#  'SR_B2',
#  'SR_B3',
#  'SR_B4',
#  'SR_B5',
#  'SR_B6',
#  'SR_B7',
#  'NDVI',
#  'elevation',
#  'percent_tree_cover',
#  'landCover','burnSeverity']]

In [8]:
# neighbors= neighbors[['SR_B1',
#  'SR_B2',
#  'SR_B3',
#  'SR_B4',
#  'SR_B5',
#  'SR_B6',
#  'SR_B7',
#  'NDVI',
#  'elevation',
#  'percent_tree_cover',
#  'landCover','burnSeverity']]

In [45]:
# clean_df = neighbors[['SR_B1',
#  'SR_B2',
#  'SR_B3',
#  'SR_B4',
#  'SR_B5',
#  'SR_B6',
#  'SR_B7',
#  'NDVI',
#  'elevation',
#  'percent_tree_cover',
#  'landCover','burnSeverity']]
for i in clean_df.columns:
    clean_df[i].fillna(value=clean_df[i].mean(), inplace=True)
X_train = copy.deepcopy(clean_df.drop(['burnSeverity'],axis=1))
scaler = preprocessing.StandardScaler().fit(X_train.values)
Xt_scaled = scaler.transform(X_train)
Xt_scaled

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:515: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


array([[-0.10924143, -0.20573535,  1.10297291, ...,  3.06821893,
        -0.70302849,  1.23064532],
       [-0.10924143, -0.20573535,  1.10297291, ...,  3.10097786,
        -0.72868873,  1.29712175],
       [-0.10924143, -0.20573535,  1.10297291, ...,  2.93718322,
        -0.77084485,  1.39683639],
       ...,
       [-0.10924143, -0.20573535, -0.90664058, ...,  0.14175472,
        -1.12642256, -1.22898254],
       [-0.10924143, -0.20573535, -0.90664058, ...,  0.14175472,
        -1.14291843, -1.22898254],
       [-0.10924143, -0.20573535, -0.90664058, ...,  0.32738865,
        -1.1594143 , -1.22898254]])

In [28]:
# y_train = neighbors['burnSeverity'].values
# clean_df = test
# for i in clean_df.columns:
#     clean_df[i].fillna(value=clean_df[i].mean(), inplace=True)
# X_train = copy.deepcopy(clean_df.drop(['burnSeverity'],axis=1))
# scaler = preprocessing.StandardScaler().fit(X_train.values)
# Xtest_scaled = scaler.transform(X_train)
# y_test = test['burnSeverity'].values

In [46]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train, x_test, y_train, y_test = train_test_split(Xt_scaled, encoded,test_size=0.2)
pipe = LogisticRegression(max_iter=600)
pipe.fit(x_train, y_train.ravel())
print("Logistic Regression: "+str(pipe.score(x_test, y_test)))
importance = pipe.coef_[0]

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Logistic Regression: 0.7941250995852233


/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [47]:
landcover = {0:'Other',1:'Developed',2:'Forest',3:'Shrub',4:'Grassland',5:'Agriculture'}

In [48]:
keep = []
for i in range(len(X_train.columns)):
    if str(X_train.columns[i])=='0' or str(X_train.columns[i])=='1' or str(X_train.columns[i])=='2' or str(X_train.columns[i])=='3' or str(X_train.columns[i])=='4' or str(X_train.columns[i])=='5':
        print(landcover[X_train.columns[i]]+": "+str(importance[i]))
    else:
        print(str(X_train.columns[i])+": "+str(importance[i]))
    if importance[i] >= 0.02 or importance[i]<=-0.02:
        keep.append(X_train.columns[i])
keep

Other: 0.05392497731838588
Developed: -0.14529018845024105
Forest: 0.20939933603161387
Shrub: -0.09672240475206412
Grassland: 0.07963187725826743
Agriculture: -0.4034186479720794
SR_B1: -0.4261466675340333
SR_B2: -0.43170828964399616
SR_B3: 6.119652611162454
SR_B4: 0.9692337387367699
SR_B5: -3.4956864263838963
SR_B6: 0.28996660589011597
SR_B7: -4.350705921005426
NDVI: 5.414847979623502
elevation: 0.5015807566288937
percent_tree_cover: -3.238424113647026


[0,
 1,
 2,
 3,
 4,
 5,
 'SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'NDVI',
 'elevation',
 'percent_tree_cover']

In [13]:
# filename = 'logistic_regression.sav'
# pickle.dump(pipe, open(filename, 'wb'))

In [14]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values.ravel())
x_train = Xt_scaled
mlp100 = MLPClassifier(max_iter=100).fit(x_train, y_train)

In [15]:
# filename = 'mlp.sav'
# pickle.dump(mlp100, open(filename, 'wb'))

In [ ]:
# %%time
# #https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/
# lab_enc = preprocessing.LabelEncoder()
# encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
# x_train, x_test = Xt_scaled, Xtest_scaled, 
# etc50 = ExtraTreesClassifier(n_estimators=50)
# etc50.fit(x_train, y_train)
# print("Extra Trees Classifier: "+str(etc50.score(x_test, y_test)))

In [16]:
#%%time
#https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train= Xt_scaled
clf = ExtraTreesClassifier(n_estimators=10)
clf.fit(x_train, y_train)
# print("Extra Trees Classifier: "+str(clf.score(x_test, y_test)))
# importance = clf.feature_importances_
# keep = []
# for i in range(len(X_train.columns)):
#     print(X_train.columns[i]+": "+str(importance[i]))
#     if importance[i] >= 0.02:
#         keep.append(X_train.columns[i])

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


ExtraTreesClassifier(n_estimators=10)

In [17]:
# filename = 'etc.sav'
# pickle.dump(clf, open(filename, 'wb'))

In [ ]:
y_pred = clf.predict(x_test)
confusion_matrix(y_test, y_pred)

In [ ]:
# filename = 'extra_trees_pickle.sav'
# pickle.dump(clf, open(filename, 'wb'))

In [ ]:
y_pred = clf.predict(x_test)
confusion_matrix(y_test, y_pred)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
  
# Calculation of Mean Squared Error (MSE)
print(mean_squared_error(y_test, y_pred))
confusion_matrix(y_test, y_pred)
accuracy_score(y_test, y_pred)

In [ ]:
%%time
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train, x_test = Xt_scaled, Xtest_scaled
xbg = XGBClassifier()
xbg.fit(x_train, y_train)
xbg.score(x_test, y_test)

In [18]:
#%%time
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train= Xt_scaled
rfc100 = RandomForestClassifier(n_estimators=100)
rfc100.fit(x_train, y_train)

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


RandomForestClassifier()

In [ ]:
y_pred = clf.predict(x_test)
confusion_matrix(y_test, y_pred)

In [19]:
%%time
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train= Xt_scaled
rfc100 = RandomForestClassifier(n_estimators=10)
rfc100.fit(x_train, y_train)

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


CPU times: user 2min 5s, sys: 1.5 s, total: 2min 7s
Wall time: 2min 8s


RandomForestClassifier(n_estimators=10)

In [25]:
# filename = 'rfc.sav'
# pickle.dump(rfc100, open(filename, 'wb'))

In [ ]:
scores

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
importance = clf.feature_importances_
keep = []
for i in range(len(X_train.columns)):
    print(X_train.columns[i]+": "+str(importance[i]))
    if importance[i] >= 0.02:
        keep.append(X_train.columns[i])
keep

In [ ]:
df['FIRE_NAME'].value_counts()

In [ ]:
temp = df[df['FIRE_NAME']=='CARR']
clean_df = temp[['burnSeverity','SR_B3','SR_B6','NDVI','elevation','percent_tree_cover','x_coord','y_coord']]
for i in clean_df.columns:
    clean_df[i]=pd.to_numeric(clean_df[i], errors='coerce')
for i in clean_df.columns:
    clean_df[i].fillna(value=clean_df[i].mean(), inplace=True)
X_train = clean_df.drop(['burnSeverity'],axis=1)
scaler = preprocessing.StandardScaler().fit(X_train.values)
X_scaled = scaler.transform(X_train)
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train = clean_df[['SR_B3','SR_B6','NDVI','elevation','percent_tree_cover','x_coord','y_coord']]
y_train = clean_df[['burnSeverity']]
clf = RandomForestClassifier()
clf.fit(x_train, y_train)
temp = df[df['FIRE_NAME']=='FRYING PAN']
clean_df = temp[['burnSeverity','SR_B3','SR_B6','NDVI','elevation','percent_tree_cover','x_coord','y_coord']]
for i in clean_df.columns:
    clean_df[i]=pd.to_numeric(clean_df[i], errors='coerce')
for i in clean_df.columns:
    clean_df[i].fillna(value=clean_df[i].mean(), inplace=True)
x_test = clean_df[['SR_B3','SR_B6','NDVI','elevation','percent_tree_cover','x_coord','y_coord']]
y_test = clean_df[['burnSeverity']]
y_pred = clf.predict(x_test)
clf.score(x_test, y_test)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
((y_pred - y_test['burnSeverity'])**2).mean()